In [38]:
%time
%load_ext autotime
%load_ext autoreload
%autoreload 2

import os
import sys
parent_dir = os.path.abspath(os.getcwd()+'/..')+'/' 
sys.path.append(parent_dir) 

import pandas as pd
import numpy as np
from copy import deepcopy

from utils.path import dir_HugeFiles
from utils.preprocessing import load
from utils.save import make_dir, save_pickle, load_pickle, auto_save_csv, print_time, auto_save_pickle

from models.fmin2 import fmin2, fmin2_model1
from models.nested_validation import *
from models.features import fixed_makedata, salvador_wrap, pretrained_wrap
from models.display import pickle2df

# models
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier

pd.set_option('display.max_colwidth', -1)

import warnings
warnings.filterwarnings("ignore")

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 9.78 µs
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
time: 41.7 ms


In [39]:
dir_save = os.path.normpath('../data/dic_20190819.pickle')
dic = load(dir_save)

amt_GI = load_pickle('../data/amt_hGI_pos.pickle')
for i, v in dic.items():
    if i in amt_GI.keys():
        v['AMT'] = amt_GI[i]
amt_ls = list(amt_GI.keys())
ls = amt_ls

exist
time: 17.3 s


### 1. word2vec and LR

In [44]:
def inner_CV(corp, medel1, method):
    '''
    should speficify the parameters here
    '''
    from models.fmin2 import p2_lr
    
    spaces = {'model1': model1, 'model2': LogisticRegression,'method': method,
              'p2': p2_lr, 'corp': corp}
    
    ''' fmin from hyperopt
    Args:
        fn: model to run, in the form of function
        space: dict of potential parameters
    Return:
        best: dict of best parameters
    ''' 
    best_space = fmin2(fn = objective, space=spaces, max_evals = 10)
    print(best_space)
    model2 = best_space['model2'](**best_space['p2'])
    return model2


'''
0. start the model selection and evaluation
'''
dfs = []
corp0 = fixed_makedata(dic, ls, tag='AMT', add_skip_thoughts='../data/X_skipthoughts_25k.pickle')
knowns = corp0.knowns
state = 2022
model1 = None
for method in [['skip','scale']]:
    corp = copy.deepcopy(corp0)
    results = []
    model2, fold_count = '', 0
    print('state', state)
    ss = StratifiedKFold(n_splits = 5, shuffle = True, random_state = state)
    for train_index, test_index in ss.split(corp.ls, corp.y):
        print('outer fold')
        print_time()
        '''
        1. model selection
        ::: model1 is for feature engineering
            e.g. model1 = pretrained_wrap(corp.knowns, 'glove-wiki-gigaword-300')
            e.g. model1 = gensim_wrap(corp.knowns, Word2Vec, params = {})
        ::: model2 is a classifier
            e.g. model2 = copy.deepcopy(classifier['default_Logistic'])
        '''
        model2 = inner_CV(corp.replace(train_index), model1, method)
        '''
        2. build X
        '''
        data = inputs_generater(corp.replace(train_index), model1, method)
        '''
        3. run and find the best prob threshold
        '''
        threshold = clf_running_search(data, model1, copy.deepcopy(model2), method)
        '''
        4. run again and evaluate on the real test set
        ''' 
        data = inputs_generater(corp.add_train_test(train_index, test_index), model1, method)
        result = clf_running(data, model1, copy.deepcopy(model2) ,method, threshold).result
        result.update({'model1': model1,'tag': corp.tag, 'method':method})
        results.append(result)
        print(result['test_f1'])

    pickle_path = auto_save_pickle(results)
    df = pickle2df(results, pickle_path)
    dfs.append(df)
auto_save_csv(pd.concat(dfs))

state 2022
outer fold
2019-08-23 16:11:40.470544
{'class_weight': 'balanced', 'solver': 'liblinear', 'penalty': 'l2', 'C': 0.5}
2019-08-23 16:11:40.797945
[0.7307692307692308, 0.7, 0.717948717948718, 0.75, 0.7594936708860761]
0.731642323920805
-0.731642323920805 -0.731642323920805
{'class_weight': 'balanced', 'solver': 'liblinear', 'penalty': 'l2', 'C': 1.0}
2019-08-23 16:11:49.525289
[0.7225806451612903, 0.691358024691358, 0.7051282051282051, 0.7455621301775148, 0.7594936708860761]
0.7248245352088888
-0.7248245352088888 -0.731642323920805
{'class_weight': 'balanced', 'solver': 'liblinear', 'penalty': 'l2', 'C': 3.0}
2019-08-23 16:11:57.580493
[0.7236842105263157, 0.6832298136645963, 0.6835443037974684, 0.738095238095238, 0.7594936708860761]
0.717609447393939
-0.717609447393939 -0.731642323920805
{'class_weight': 'balanced', 'solver': 'liblinear', 'penalty': 'l2', 'C': 1000}
2019-08-23 16:12:06.559060
[0.6891891891891893, 0.6666666666666667, 0.6835443037974684, 0.7337278106508877, 0.74

test_f1 test_precision test_recall test_accuracy train_f1 train_precision  \
0  0.79    0.803          0.779       0.788         0.941    0.949            

  train_recall train_accuracy model1  \
0  0.933        0.94           None    

                                                                                                                                                                                                                                                                               model2  \
0  LogisticRegression(C=0.05, class_weight='balanced', dual=False,\n          fit_intercept=True, intercept_scaling=1, max_iter=100,\n          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,\n          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)   

          method  tag                                  pickle_path  
0  [skip, scale]  AMT  ../pickle/2019-08-23 16:18:27.590588.pickle

save to ../csv/2019-08-23 16:18:27.648576.csv


test_f1 test_precision test_recall test_accuracy train_f1 train_precision  \
0  0.79    0.803          0.779       0.788         0.941    0.949            

  train_recall train_accuracy model1  \
0  0.933        0.94           None    

                                                                                                                                                                                                                                                                               model2  \
0  LogisticRegression(C=0.05, class_weight='balanced', dual=False,\n          fit_intercept=True, intercept_scaling=1, max_iter=100,\n          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,\n          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)   

          method  tag                                  pickle_path  
0  [skip, scale]  AMT  ../pickle/2019-08-23 16:18:27.590588.pickle

time: 6min 59s


### LGBM

In [45]:
def inner_CV(corp, medel1, method):
    '''
    should speficify the parameters here
    '''
    from models.fmin2 import p2_lgbm
    
    spaces = {'model1': model1, 'model2': LGBMClassifier,'method': method,
              'p2': p2_lgbm, 'corp': corp}
    
    ''' fmin from hyperopt
    Args:
        fn: model to run, in the form of function
        space: dict of potential parameters
    Return:
        best: dict of best parameters
    ''' 
    best_space = fmin2(fn = objective, space=spaces, max_evals = 20)
    print(best_space)
    model2 = best_space['model2'](**best_space['p2'])
    return model2


'''
0. start the model selection and evaluation
'''
dfs = []
corp0 = fixed_makedata(dic, ls, tag='AMT', add_skip_thoughts='../data/X_skipthoughts_25k.pickle')
knowns = corp0.knowns
state = 2022
model1 = None
for method in [['skip','scale']]:
    corp = copy.deepcopy(corp0)
    results = []
    model2, fold_count = '', 0
    print('state', state)
    ss = StratifiedKFold(n_splits = 5, shuffle = True, random_state = state)
    for train_index, test_index in ss.split(corp.ls, corp.y):
        print('outer fold')
        print_time()
        '''
        1. model selection
        ::: model1 is for feature engineering
            e.g. model1 = pretrained_wrap(corp.knowns, 'glove-wiki-gigaword-300')
            e.g. model1 = gensim_wrap(corp.knowns, Word2Vec, params = {})
        ::: model2 is a classifier
            e.g. model2 = copy.deepcopy(classifier['default_Logistic'])
        '''
        model2 = inner_CV(corp.replace(train_index), model1, method)
        '''
        2. build X
        '''
        data = inputs_generater(corp.replace(train_index), model1, method)
        '''
        3. run and find the best prob threshold
        '''
        threshold = clf_running_search(data, model1, copy.deepcopy(model2), method)
        '''
        4. run again and evaluate on the real test set
        ''' 
        data = inputs_generater(corp.add_train_test(train_index, test_index), model1, method)
        result = clf_running(data, model1, copy.deepcopy(model2) ,method, threshold).result
        result.update({'model1': model1,'tag': corp.tag, 'method':method})
        results.append(result)
        print(result['test_f1'])

    pickle_path = auto_save_pickle(results)
    df = pickle2df(results, pickle_path)
    dfs.append(df)
auto_save_csv(pd.concat(dfs))

state 2022
outer fold
2019-08-23 16:20:21.896676
{'class_weight': 'balanced', 'boosting': 'gbrt', 'num_leaves': 256, 'max_depth': 16, 'learning_rate': 0.15, 'gamma': 0.65, 'n_estimators': 2000, 'lambda_l2': 1, 'feature_fraction': 0.5, 'bagging_fraction': 0.75, 'bagging_freq': 5, 'subsample': 0.7}
2019-08-23 16:20:22.228319
[0.7515923566878981, 0.75, 0.7785234899328859, 0.8313253012048193, 0.7894736842105262]
0.780182966407226
-0.780182966407226 -0.780182966407226
{'class_weight': 'balanced', 'boosting': 'gbrt', 'num_leaves': 512, 'max_depth': 8, 'learning_rate': 0.15, 'gamma': 0.55, 'n_estimators': 2000, 'lambda_l2': 1, 'feature_fraction': 0.75, 'bagging_fraction': 0.5, 'bagging_freq': 5, 'subsample': 0.7}
2019-08-23 16:32:04.146564
[0.7560975609756098, 0.717948717948718, 0.7712418300653595, 0.8, 0.7922077922077922]
0.7674991802394958
-0.7674991802394958 -0.780182966407226
{'class_weight': 'balanced', 'boosting': 'gbrt', 'num_leaves': 64, 'max_depth': 8, 'learning_rate': 0.1, 'gamma': 

test_f1 test_precision test_recall test_accuracy train_f1 train_precision  \
0  0.79    0.818          0.767       0.793         1        1                

  train_recall train_accuracy model1  \
0  1            1              None    

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   model2  \
0  LGBMClassifier(bagging_fraction=0.75, bagging_freq=5, boosting='gbrt',\n        boosting_type='gbdt', class_weight='balanced',\n        colsample_bytree=1.0, feature_fraction=0.75, gamma=0.45,\n        importance_type='split', lambda_l2=0.1, learning_rate=0.15,\n        max_depth=8, min_child_samples=20, min_child_weight=0.001,\n        min_split_gain=0.0, n_estimators=2000, n_jobs=-1, num_leaves=32,\n        objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,\n        silent=True, subsample=1, subsample_for_bin=200000,\n        subsample_freq=0)   

          method  tag                                  pickle_path  
0  [skip, scale]  AMT  ../pickle/2019-08-24 11:24:27.687884.pickle

save to ../csv/2019-08-24 11:24:28.152424.csv


test_f1 test_precision test_recall test_accuracy train_f1 train_precision  \
0  0.79    0.818          0.767       0.793         1        1                

  train_recall train_accuracy model1  \
0  1            1              None    

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   model2  \
0  LGBMClassifier(bagging_fraction=0.75, bagging_freq=5, boosting='gbrt',\n        boosting_type='gbdt', class_weight='balanced',\n        colsample_bytree=1.0, feature_fraction=0.75, gamma=0.45,\n        importance_type='split', lambda_l2=0.1, learning_rate=0.15,\n        max_depth=8, min_child_samples=20, min_child_weight=0.001,\n        min_split_gain=0.0, n_estimators=2000, n_jobs=-1, num_leaves=32,\n        objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,\n        silent=True, subsample=1, subsample_for_bin=200000,\n        subsample_freq=0)   

          method  tag                                  pickle_path  
0  [skip, scale]  AMT  ../pickle/2019-08-24 11:24:27.687884.pickle

time: 19h 4min 18s
